In [ ]:
import pandas as pd
import numpy as np
import gridstatusio
from datetime import datetime, timedelta
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [2]:
start_date_str = "2023-07-01"
next_date = datetime.strptime(start_date_str, "%Y-%m-%d") + timedelta(days=1)
new_date_str = next_date.strftime("%Y-%m-%d")

In [ ]:
class statistical_model():
    def __init__(self):
        self.client = gridstatusio.GridStatusClient(api_key = '5d9fe9cd74854c598de04e2c9f3a203b')
        self.kernel = RBF(length_scale=10, length_scale_bounds=(1e-2, 1e2))
        self.model = GaussianProcessRegressor(kernel=self.kernel, n_restarts_optimizer=10)
        self.start_date = '2023-07-01'
        self.end_date = '2023-09-01'

    def get_lmps_statusgrid(self, start_date:str, end_date:str) -> pd.DataFrame:
        self.lmps = self.client.get_dataset(
                                    dataset = "ercot_lmp_by_settlement_point",
                                    start = start_date,
                                    end = end_date,    
                                    timezone = "US/Central",
                                    )
        self.lmps

    def get_capacity_forecast_statusgrid(self, start_date:str, end_date:str) -> pd.DataFrame:
        self.capacity_forecast = self.client.get_dataset(
                                    dataset = "ercot_capacity_forecast",
                                    start = start_date,
                                    end = end_date,    
                                    timezone = "US/Central",
                                    )
        self.capacity_forecast

    
    def get_load_forecast_statusgrid(self, start_date:str, end_date:str) -> pd.DataFrame:
        self.load_forecast = self.client.get_dataset(
                                    dataset = "ercot_load_by_forecast_zone",
                                    start = start_date,
                                    end = end_date,    
                                    timezone = "US/Central",
                                    )
        self.load_forecast

    
    
    def get_lmps_csv(self, file) -> pd.DataFrame:
        self.lmps = pd.read_csv(file)
        self.lmps.set_index('Date', inplace= True, drop=True)
        self.lmps = self.preprocess_data(self.lmps)

    def get_load_forecast_csv(self, file) -> pd.DataFrame:
        self.demand_forecast = pd.read_csv(file)
        self.demand_forecast.set_index('Date', inplace= True, drop=True)
        self.demand_forecast = self.preprocess_data(self.demand_forecast)

    def get_capacity_forecast_csv(self, file) -> pd.DataFrame:
        self.demand_forecast = pd.read_csv(file)
        self.demand_forecast.set_index('Date', inplace= True, drop=True)
        self.demand_forecast = self.preprocess_data(self.demand_forecast)

    def preprocess_data(self, df):
        df.dropna(inplace = True)
        return df
        
    def train_model(self, x, y):
        self.model.fit(x, y)
    
    def forecast(self, x):
        '''
        given a set of forecast demand 
        '''
        pred, st_dev = self.model.predict(x)
        return pred, st_dev
        

In [ ]:
first_model = statistical_model()
first_model.get_lmps_csv("data/nodal_lmps.csv")
first_model.get_demand_forecast_csv("data/zonal_demand_forecast.csv")
first_model.train_model()

               GS-PIQ: ERCOT COAST Demand Forecast (Mod) Multi-Day (52627)-MW  \
Date                                                                            
7/1/2023 0:00                                              15070                
7/1/2023 1:00                                              14516                
7/1/2023 2:00                                              14009                
7/1/2023 3:00                                              13665                
7/1/2023 4:00                                              13353                

               GS-PIQ: ERCOT EAST Demand Forecast (Mod) Multi-Day (52631)-MW  \
Date                                                                           
7/1/2023 0:00                                               2032               
7/1/2023 1:00                                               1929               
7/1/2023 2:00                                               1822               
7/1/2023 3:00                   

c:\Users\luisv\ML-AI\statistical_lmps\stat_env\lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [10]:
x = first_model.demand_forecast.iloc[0:2]
print(x)
pred, std = first_model.forecast(x)

print(pred)

               GS-PIQ: ERCOT COAST Demand Forecast (Mod) Multi-Day (52627)-MW  \
Date                                                                            
7/1/2023 0:00                                              15070                
7/1/2023 1:00                                              14516                

               GS-PIQ: ERCOT EAST Demand Forecast (Mod) Multi-Day (52631)-MW  \
Date                                                                           
7/1/2023 0:00                                               2032               
7/1/2023 1:00                                               1929               

               GS-PIQ: ERCOT FAR WEST Demand Forecast (Mod) Multi-Day (52635)-MW  \
Date                                                                               
7/1/2023 0:00                                               5914                   
7/1/2023 1:00                                               5807                   

               GS